### Image Quality Assessment
-----------------------------------
#### Steps:
**1:** Read in all the images from the server.

**2:** Break the images into smaller categories. This can be accomplished via a K-Means classifier or possibly by running the images through a different CNN beforehand. These categories are then stored and used later. 

**3:** Determine labels based on the metadata stored in the pictures. For many of the 2017 pictures there is exif, IPTC, and XMP. The differences are as follows:

* exif - Stored by camera at the time of the picture. This includes GPS location, pixel count, zoom, etc.
* IPTC - Metadata added via an editor. This is generally an older bulk metadata system. While still used it is getting phased out by XMP.
* XMP - Metadata that can be applied as an "xml"-like file or added in headers and footers of images. It is more clearly defined than IPTC and significantly newer. It also allows for more metadata to be stored. 

For both IPTC and XMP, they contain information we can use as labels. We have preference tags (code 221) where information has been stored determining which pictures have been preferenced before our work began. In addition to this, the Reynolds Journalism School has informed us that they have color classed about half of the images via PhotoMechanic which is stored in the IPTC and XMP data. These color classes correspond to a quality assessment on a rough range of 0-9. Once labels have been determined we can begin training.

**4:** Import VGG-16. This is a very successful pretrained classification model that has shown promise in previous studies attempting similar techniques as us, most notibly NIMA (Neural IMage Assessment) by the perceptron team at Google.AI in 2015.

**5:** Once the pretrained model has been imported we can pull the fully connected component off the top and replace it with our own. Initially we will replace it with one total classifier to determine quality but after proof of concept has been established we can supply two different classifiers (1 for technical assessment and 1 for aesthetics). These classifiers shall be constant for each category of picture while just changing the loaded weights for the system. This allows us to adjust our idea of what quality is based on the category it is in. 

**6:** After the models have provided a technical and aesthetic score we can use the Choquet Integral to aggregate the data together and get a measurement of uncertainty. This shall provide us with a rating and a range both of which can be used to rank photos in a set. 

In [35]:
#standard ML/Image Processing imports
import numpy as np
import math, pandas
import matplotlib.image as mpimg
from libxmp import XMPFiles, consts
# import libxmp

from PIL import Image

#pytorch imports
import torch
import torch.optim as optim
import torchvision.models as models

from torch import nn
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms

# no one likes irrelevant warnings
import warnings  
warnings.filterwarnings('ignore')

# root directory where the images are stored
data_dir = "../../../../../mnt/md0/mysql-dump-economists/Archives"#/Fall"#/Dump"

At this point, we have created all our global variables necessary for the classifier and imported all necessary libraries. Next we need to determine the images under test. Once we have gotten access to GPUs we can scale this up.

The class below extends PyTorch's native ImageFolder class, with an adjustment to store/retrieve file paths. The native ImageFolder class allows us to iterate through and pull all files from inside a home directory.

There is also a function defined below which is just an optional "helper" function to help us get an idea of what our data looks like so that we know what to resize it to (we do not want to make the dimensions of the resize to be bigger than any of the pictures)

In [118]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        print(tuple_with_path)
        return tuple_with_path
ratings = None
class ImageFolderWithPathsAndRatings(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPathsAndRatings, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
#         return tuple_with_path
        # set rating
        try:
            tuple_with_path_and_rating = (tuple_with_path + (ratings[index],))
        except:
            tuple_with_path_and_rating = (tuple_with_path + (torch.FloatTensor([0]),))
        return tuple_with_path_and_rating

def find_size_bounds(limit_num_pictures=None):
    """ Will print and return min/max width/height of pictures in the dataset 
    :param limit_num_pictures - limits the number of pictures analyzed if you purposefully 
        want to work with a smaller dataset
    """
    data = ImageFolderWithPaths(data_dir)
    print(data[0][0].size)
    max_h = (data[0][0]).size[1]
    min_h = data[0][0].size[1]
    max_w = data[0][0].size[0]
    min_w = data[0][0].size[0]
    try:
        for (i, pic) in enumerate(data):
            #if we are limiting pics
            if limit_num_pictures:
                if i > limit_num_pictures:
                    break
            print(pic[0].size) # print all size dimensions
            
            # check width records
            if pic[0].size[0] > max_w:
                max_w = pic[0].size[0]
            elif pic[0].size[1] < min_w:
                min_w = pic[0].size[0]

            # check height records
            if pic[0].size[1] > max_h:
                max_h = pic[0].size[1]
            elif pic[0].size[1] < min_h:
                min_h = pic[0].size[1]
    except Exception as e:
        print(e)
        print("error occurred on pic {} number {}".format(pic, i))

    print("Max/min width: {} {}".format(max_w, min_w))
    print("Max/min height: {} {}".format(max_h, min_h))
    return min_w, max_w, min_h, max_h
#find_size_bounds(limit_num_pictures=1000)

### Reading and Interpreting Metadata

There is a lot of data stored in each of the images. It is stored in 3 types of metadata extensions. They are:
1. XMP
2. Exif
3. IPTC

Because we mostly care about the color classes and whether images are "tagged" or not we need edited in data. This data is supplied using Photomechanic or Photoshop and looks different to an image. Because Exif is the data the camera collected at the time the picture was taken, this means that this data is stored in the XMP and IPTC data. IPTC (International Press Telecommunications Council) Metadata is an older varient where XMP is newer and can be stored either in a sidecar file or in the image itself. This may make a difference as we work backward in year over our images. But starting in 2017, XMP is easier to work with so we will use it as the baseline. Metadata tags look like the following:

##### Steps to find XMP metadata in image:
1. Read in image via file path
2. Open the image in read binary mode
3. Search the image to find the '<x:xmpmeta' tag and read to the '</x:xmpmeta' closing tag
4. Add padding for last close xmp tag
5. Print out the xmp data

From here the most useful 3 data tags are located near the top. They are:
```
photomechanic:ColorClass="0"
photomechanic:Tagged="False"
photomechanic:Prefs="0:0:0:008714"
```
Where tagged is whether the image is tagged at all, the prefs a base 64 string that contains specially designated preferences, and the final color class is the color code assigned. The color code exists on ~20% of all images in the dataset and spans a range of 1-8.

In [129]:
# load data and apply the transforms on contained pictures

_transform = transforms.Compose([transforms.ToTensor()])

data = ImageFolderWithPaths(data_dir, transform=_transform)

data_loader = torch.utils.data.DataLoader(data)#, num_workers=4)

limit_num_pictures = 200

rated_indices = []
ratings = []
bad_indices = []
try:
    for i, data in enumerate(data_loader):
        if limit_num_pictures:
            if i > limit_num_pictures:
                break
        inputs, labels, path = data
        path = path[0].rstrip()
#         xmp = XMPFiles( file_path=path, open_forupdate=True ).get_xmp()
#         print(xmp)
#         print(xmp.get_property(consts.XMP_NS_DC, 'ColorClass' ))
        with open(path, "rb") as f:
            img = f.read()
        img_string = str(img)
        xmp_start = img_string.find('photomechanic:ColorClass')
        xmp_end = img_string.find('photomechanic:Tagged')
        if xmp_start != xmp_end:
            xmp_string = img_string[xmp_start:xmp_end]
            if xmp_string[26] != "0":
                print(xmp_string[26] + " " + str(path) + "\n\n")
                rated_indices.append(i)
                ratings.append(xmp_string[26])
            else:
                ratings.append(0)
                bad_indices.append(i)
except Exception as e:
    print("There was an error on image #{}: {}".format(i, e))
print(counter)


(tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0353,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0039, 0.0471,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.3451, 0.3216, 0.3137,  ..., 0.4706, 0.4392, 0.3961],
         [0.3843, 0.3216, 0.3373,  ..., 0.5216, 0.4784, 0.4235],
         [0.3725, 0.3255, 0.3098,  ..., 0.5020, 0.4431, 0.3843]],

        [[0.0000, 0.0000, 0.0078,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0353,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0039, 0.0471,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.3451, 0.3216, 0.3137,  ..., 0.4706, 0.4392, 0.3961],
         [0.3843, 0.3216, 0.3373,  ..., 0.5216, 0.4784, 0.4235],
         [0.3725, 0.3255, 0.3098,  ..., 0.5020, 0.4431, 0.3843]],

        [[0.0000, 0.0000, 0.0078,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0353,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0039, 0.0471,  ..., 0.0000, 0.0000, 0

(tensor([[[0.2314, 0.2588, 0.2627,  ..., 0.3804, 0.3608, 0.3608],
         [0.2275, 0.2392, 0.2392,  ..., 0.3725, 0.3608, 0.3529],
         [0.2314, 0.2275, 0.2157,  ..., 0.3804, 0.3608, 0.3490],
         ...,
         [0.6275, 0.6510, 0.6706,  ..., 0.4392, 0.4549, 0.4588],
         [0.6431, 0.6706, 0.6824,  ..., 0.4784, 0.4863, 0.4863],
         [0.6196, 0.6471, 0.6588,  ..., 0.4784, 0.4863, 0.4863]],

        [[0.1569, 0.1765, 0.1804,  ..., 0.4745, 0.4549, 0.4431],
         [0.1569, 0.1647, 0.1569,  ..., 0.4745, 0.4549, 0.4431],
         [0.1608, 0.1569, 0.1412,  ..., 0.4824, 0.4667, 0.4510],
         ...,
         [0.7059, 0.7294, 0.7451,  ..., 0.3765, 0.3647, 0.3451],
         [0.7216, 0.7490, 0.7529,  ..., 0.3961, 0.3686, 0.3373],
         [0.6980, 0.7255, 0.7294,  ..., 0.3961, 0.3686, 0.3373]],

        [[0.1294, 0.1569, 0.1608,  ..., 0.2314, 0.2196, 0.2118],
         [0.1176, 0.1373, 0.1373,  ..., 0.2275, 0.2275, 0.2314],
         [0.1137, 0.1176, 0.1137,  ..., 0.2431, 0.2588, 0

(tensor([[[0.0980, 0.0980, 0.0941,  ..., 0.0039, 0.0353, 0.0824],
         [0.1098, 0.1098, 0.0941,  ..., 0.0314, 0.0392, 0.0667],
         [0.1020, 0.1020, 0.0902,  ..., 0.0275, 0.0431, 0.0588],
         ...,
         [0.1098, 0.1020, 0.0980,  ..., 0.1922, 0.1882, 0.1686],
         [0.1137, 0.1059, 0.0980,  ..., 0.1647, 0.1882, 0.1765],
         [0.1255, 0.1176, 0.1059,  ..., 0.1608, 0.1882, 0.1804]],

        [[0.0980, 0.0980, 0.0941,  ..., 0.0039, 0.0353, 0.0824],
         [0.1098, 0.1098, 0.0941,  ..., 0.0314, 0.0392, 0.0667],
         [0.1020, 0.1020, 0.0902,  ..., 0.0275, 0.0431, 0.0588],
         ...,
         [0.1098, 0.1020, 0.0980,  ..., 0.1922, 0.1882, 0.1686],
         [0.1137, 0.1059, 0.0980,  ..., 0.1647, 0.1882, 0.1765],
         [0.1255, 0.1176, 0.1059,  ..., 0.1608, 0.1882, 0.1804]],

        [[0.0980, 0.0980, 0.0941,  ..., 0.0039, 0.0353, 0.0824],
         [0.1098, 0.1098, 0.0941,  ..., 0.0314, 0.0392, 0.0667],
         [0.1020, 0.1020, 0.0902,  ..., 0.0275, 0.0431, 0

(tensor([[[0.6431, 0.6863, 0.6549,  ..., 0.5255, 0.6196, 0.5569],
         [0.6157, 0.6902, 0.6824,  ..., 0.5961, 0.5373, 0.4784],
         [0.6275, 0.6863, 0.6824,  ..., 0.6314, 0.5451, 0.4784],
         ...,
         [0.3490, 0.4235, 0.3412,  ..., 0.4353, 0.2863, 0.1922],
         [0.4314, 0.3686, 0.3137,  ..., 0.5333, 0.5020, 0.4000],
         [0.3765, 0.3686, 0.3294,  ..., 0.4980, 0.5922, 0.3843]],

        [[0.6431, 0.6863, 0.6549,  ..., 0.5255, 0.6196, 0.5569],
         [0.6157, 0.6902, 0.6824,  ..., 0.5961, 0.5373, 0.4784],
         [0.6275, 0.6863, 0.6824,  ..., 0.6314, 0.5451, 0.4784],
         ...,
         [0.3490, 0.4235, 0.3412,  ..., 0.4353, 0.2863, 0.1922],
         [0.4314, 0.3686, 0.3137,  ..., 0.5333, 0.5020, 0.4000],
         [0.3765, 0.3686, 0.3294,  ..., 0.4980, 0.5922, 0.3843]],

        [[0.6431, 0.6863, 0.6549,  ..., 0.5255, 0.6196, 0.5569],
         [0.6157, 0.6902, 0.6824,  ..., 0.5961, 0.5373, 0.4784],
         [0.6275, 0.6863, 0.6824,  ..., 0.6314, 0.5451, 0

8 ../../../../../mnt/md0/mysql-dump-economists/Archives/1999/Fall/Dump/Coffin, Jessica/1999_087619color_JC.JPG


(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 0, '../../../../../mnt/md0/mysql-dump-economists/Archives/1999/Fall/Dump/Coffin, Jessica/1999_08762

...

1 ../../../../../mnt/md0/mysql-dump-economists/Archives/1999/Fall/Dump/Cook, Jason/1999_15ice2_JC.JPG


(tensor([[[0.0471, 0.0549, 0.0784,  ..., 0.0549, 0.0549, 0.0549],
         [0.0078, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0431, 0.0078, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.6039, 0.6784, 0.7725,  ..., 0.9216, 0.6980, 0.7333],
         [0.5922, 0.5765, 0.6157,  ..., 0.8392, 0.8745, 0.7137],
         [0.6118, 0.6000, 0.5333,  ..., 0.7882, 0.8196, 0.8314]],

        [[0.0471, 0.0549, 0.0784,  ..., 0.0549, 0.0549, 0.0549],
         [0.0078, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0431, 0.0078, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.6039, 0.6784, 0.7725,  ..., 0.9216, 0.6980, 0.7333],
         [0.5922, 0.5765, 0.6157,  ..., 0.8392, 0.8745, 0.7137],
         [0.6118, 0.6000, 0.5333,  ..., 0.7882, 0.8196, 0.8314]],

        [[0.0471, 0.0549, 0.0784,  ..., 0.0549, 0.0549, 0.0549],
         [0.0078, 

1 ../../../../../mnt/md0/mysql-dump-economists/Archives/1999/Fall/Dump/Cook, Jason/1999_31musoccer_JC.JPG


(tensor([[[1.0000, 1.0000, 1.0000,  ..., 0.0118, 0.0196, 0.0157],
         [1.0000, 1.0000, 1.0000,  ..., 0.0039, 0.0039, 0.0039],
         [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.2863, 0.3137, 0.2980,  ..., 0.2588, 0.2627, 0.1961],
         [0.2784, 0.2941, 0.2784,  ..., 0.2549, 0.2667, 0.2235],
         [0.2863, 0.3059, 0.2784,  ..., 0.2392, 0.2784, 0.2706]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.0118, 0.0196, 0.0157],
         [1.0000, 1.0000, 1.0000,  ..., 0.0039, 0.0039, 0.0039],
         [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.2863, 0.3137, 0.2980,  ..., 0.2588, 0.2627, 0.1961],
         [0.2784, 0.2941, 0.2784,  ..., 0.2549, 0.2667, 0.2235],
         [0.2863, 0.3059, 0.2784,  ..., 0.2392, 0.2784, 0.2706]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.0118, 0.0196, 0.0157],
         [1.00

(tensor([[[0.9176, 0.8784, 0.8745,  ..., 0.2627, 0.2314, 0.2706],
         [0.8275, 0.8275, 0.8863,  ..., 0.2980, 0.2588, 0.2980],
         [0.7333, 0.7843, 0.8784,  ..., 0.2745, 0.2863, 0.3098],
         ...,
         [0.5922, 0.5686, 0.5569,  ..., 0.3294, 0.3725, 0.3843],
         [0.6431, 0.5922, 0.5490,  ..., 0.3647, 0.3804, 0.4196],
         [0.6196, 0.5725, 0.5373,  ..., 0.3961, 0.4039, 0.4157]],

        [[0.8745, 0.8863, 0.8784,  ..., 0.2706, 0.2314, 0.2784],
         [0.8275, 0.8353, 0.9098,  ..., 0.2941, 0.2588, 0.2980],
         [0.7686, 0.7725, 0.9176,  ..., 0.2824, 0.3059, 0.3098],
         ...,
         [0.5843, 0.5725, 0.5647,  ..., 0.3490, 0.3647, 0.3922],
         [0.6471, 0.5961, 0.5490,  ..., 0.3725, 0.3765, 0.4157],
         [0.5961, 0.5569, 0.5412,  ..., 0.3922, 0.4039, 0.4118]],

        [[0.9059, 0.8824, 0.8549,  ..., 0.2667, 0.2235, 0.2667],
         [0.8353, 0.8157, 0.8706,  ..., 0.2784, 0.2510, 0.2980],
         [0.7647, 0.7529, 0.8863,  ..., 0.2706, 0.2902, 0

(tensor([[[0.0118, 0.0078, 0.0039,  ..., 0.2431, 0.2667, 0.2039],
         [0.0118, 0.0078, 0.0039,  ..., 0.2706, 0.2667, 0.2039],
         [0.0118, 0.0078, 0.0039,  ..., 0.2980, 0.2824, 0.2196],
         ...,
         [0.1137, 0.1020, 0.0784,  ..., 0.8980, 0.9059, 0.7804],
         [0.1373, 0.1255, 0.1020,  ..., 0.9059, 0.9569, 0.8196],
         [0.1412, 0.1294, 0.1059,  ..., 0.7765, 0.8314, 0.7059]],

        [[0.0118, 0.0078, 0.0039,  ..., 0.2431, 0.2667, 0.2039],
         [0.0118, 0.0078, 0.0039,  ..., 0.2706, 0.2667, 0.2039],
         [0.0118, 0.0078, 0.0039,  ..., 0.2980, 0.2824, 0.2196],
         ...,
         [0.1137, 0.1020, 0.0784,  ..., 0.8980, 0.9059, 0.7804],
         [0.1373, 0.1255, 0.1020,  ..., 0.9059, 0.9569, 0.8196],
         [0.1412, 0.1294, 0.1059,  ..., 0.7765, 0.8314, 0.7059]],

        [[0.0118, 0.0078, 0.0039,  ..., 0.2431, 0.2667, 0.2039],
         [0.0118, 0.0078, 0.0039,  ..., 0.2706, 0.2667, 0.2039],
         [0.0118, 0.0078, 0.0039,  ..., 0.2980, 0.2824, 0

(tensor([[[0.8706, 0.8118, 0.8275,  ..., 0.6745, 0.7137, 0.7412],
         [0.8824, 0.8275, 0.8392,  ..., 0.6941, 0.6980, 0.7373],
         [0.8863, 0.8431, 0.8588,  ..., 0.6980, 0.6784, 0.7412],
         ...,
         [0.3882, 0.3412, 0.3569,  ..., 0.7294, 0.7176, 0.7373],
         [0.3922, 0.3451, 0.3529,  ..., 0.7373, 0.7176, 0.7529],
         [0.4000, 0.3373, 0.3412,  ..., 0.7294, 0.7255, 0.7647]],

        [[0.8706, 0.8118, 0.8275,  ..., 0.6745, 0.7137, 0.7412],
         [0.8824, 0.8275, 0.8392,  ..., 0.6941, 0.6980, 0.7373],
         [0.8863, 0.8431, 0.8588,  ..., 0.6980, 0.6784, 0.7412],
         ...,
         [0.3882, 0.3412, 0.3569,  ..., 0.7294, 0.7176, 0.7373],
         [0.3922, 0.3451, 0.3529,  ..., 0.7373, 0.7176, 0.7529],
         [0.4000, 0.3373, 0.3412,  ..., 0.7294, 0.7255, 0.7647]],

        [[0.8706, 0.8118, 0.8275,  ..., 0.6745, 0.7137, 0.7412],
         [0.8824, 0.8275, 0.8392,  ..., 0.6941, 0.6980, 0.7373],
         [0.8863, 0.8431, 0.8588,  ..., 0.6980, 0.6784, 0

Below we split the data into training and testing sets, perform our defined transform to resize and crop pictures, and define the loaders that will load all the pictures

In [130]:
# Define our data transforms to get all our images the same size
_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

valid_size = 0.2 # percentage of data to use for test set

# load data and apply the transforms on contained pictures
train_data = ImageFolderWithPathsAndRatings(data_dir, transform=_transform)
test_data = ImageFolderWithPathsAndRatings(data_dir, transform=_transform)   

num_pictures = len(train_data)
print("Number of pictures in subdirectories: {}".format(num_pictures))

# Shuffle pictures and split training set
indices = list(range(num_pictures))
# print("Head of indices: {}".format(indices[:10]))

split = int(np.floor(valid_size * num_pictures))
print("Split index: {}".format(split))

# may be unnecessary with the choice of sampler below
#     np.random.shuffle(indices)
#     print("Head of shuffled indices: {}".format(indices[:10]))

train_idx, test_idx = rated_indices, bad_indices#indices[split:], indices[:split]
print("Size of training set: {}, size of test set: {}".format(len(train_idx), len(test_idx)))

# Define samplers that sample elements randomly without replacement
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

# Define data loaders, which allow batching and shuffling the data
train_loader = torch.utils.data.DataLoader(train_data,
               sampler=train_sampler, batch_size=1)#, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data,
               sampler=test_sampler, batch_size=1)#, num_workers=4)


Number of pictures in subdirectories: 106388
Split index: 21277
Size of training set: 81, size of test set: 120


#### Download the VGG16 Neural Net

We will now load the VGG16 model from PyTorch's set of pretrained models. 

In [131]:
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device that will be used: {}".format(device))

# we load the pretrained model, the argument pretrained=True implies to load the ImageNet 
#     weights for the pre-trained model
vgg16 = models.vgg16(pretrained=True)
vgg16.to(device) # loads the model onto the device (CPU or GPU)

# change the number of classes 
vgg16.classifier[6].out_features = 10

# vgg16 # print our model

Device that will be used: cpu


#### Freezing Convolution Weights

One important thing to notice here is that the classifier model is classifying 1000 classes. You can observe the very last Linear block to confirm that. But we need to classify the images into 10 classes only. So, we will change that. Also, we will freeze all the weights of the convolutional blocks. The model as already learned many features from the ImageNet dataset. So, freezing the Conv2d() weights will make the model to use all those pre-trained weights. This is the part that really justifies the term transfer learning.

The following block of code makes the necessary changes for the 10 class classification along with freezing the weights.

#### Building Fully Connected Classifier
After we have VGG-16 set up we need to build and add the fully connected layer to it. That is accomplished below.

In [139]:
for param in vgg16.parameters():
    param.requires_grad = False #freeze all convolution weights
network = list(vgg16.classifier.children())[:-1] #remove fully connected layer
network.extend([nn.Linear(4096, 9)]) #add new layer of 4096->100 (rating scale with 1 decimal - similar to 1 hot encoding)
vgg16.classifier = nn.Sequential(*network)

criterion = nn.CrossEntropyLoss() # loss function
optimizer = optim.SGD(vgg16.parameters(), lr=0.4, momentum=0.9) # optimizer

vgg16 #print out the model to ensure our network is correct

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Now that we have the new layer added and the rest frozen, we must train the new set with our below code to fit the model

In [142]:
vgg16.train() # set model to training model
num_epochs = 2 
training_loss = 0
training_accuracy = 0
for epoch in range(num_epochs):
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(train_loader,0):
        if limit_num_pictures:
            if i > limit_num_pictures:
                break
        inputs, _, path, label = data
        label = torch.LongTensor([int(labels[0])])
        print(torch.LongTensor([int(labels[0])]))
        optimizer.zero_grad()
        output = vgg16(inputs)
        loss = criterion(output, torch.LongTensor([int(label[0])]))
        running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        num_correct += (preds == label).sum().item()
        loss.backward()
        optimizer.step()
    
        if i % 2000 == 1999:
            print(running_loss/2000)
#             running_loss = 0
        print("Completed training output for image #{}: {}".format(i, output))
    training_loss = running_loss/len(train_loader.dataset)
    training_accuracy = 100 * num_correct/len(train_loader.dataset)
    print("Training accuracy: {}, Training loss: {}".format(accuracy, loss))

tensor([8])
Completed training output for image #0: tensor([[-0.9745,  0.4672,  0.2175,  0.6526,  0.1518,  0.5020,  0.7406, -0.4208,
         -0.8153]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #1: tensor([[ -4.8156, -22.1246, -16.9352, -26.0014, -15.9560, -22.4667, -28.5680,
          -8.9052, 144.9784]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #2: tensor([[ -6.4374, -28.7535, -22.9905, -34.6333, -21.0564, -30.7002, -39.4361,
         -12.2308, 197.9461]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #3: tensor([[ -40.8915, -171.4212, -133.9384, -206.4805, -125.9040, -177.5099,
         -226.1495,  -69.9773, 1151.4611]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #4: tensor([[ -16.4651,  -76.4534,  -59.5834,  -91.1607,  -55.1547,  -79.7034,
         -100.6981,  -30.9266,  511.5766]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #5: tens

tensor([8])
Completed training output for image #43: tensor([[ -55.6328, -236.3696, -184.4224, -285.5660, -173.1291, -246.1291,
         -311.9068,  -98.0519, 1589.7456]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #44: tensor([[ -37.6800, -159.3545, -124.1114, -190.9990, -116.4767, -164.2470,
         -209.6424,  -64.9769, 1067.1173]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #45: tensor([[ -65.9308, -276.3705, -215.1231, -332.4666, -201.6192, -286.3913,
         -363.3597, -113.7005, 1855.4167]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #46: tensor([[ -85.4133, -362.1641, -282.1246, -436.7682, -263.9467, -375.3696,
         -476.2931, -148.9134, 2432.3447]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #47: tensor([[ -85.3235, -359.2035, -279.2904, -431.6129, -262.2700, -372.7018,
         -472.4102, -147.3113, 2412.2654]], grad_fn=<AddmmBackward>)
tensor([8])
Com

Completed training output for image #4: tensor([[ -23.1992,  -96.6828,  -74.8350, -116.5420,  -70.7761, -100.0470,
         -127.9209,  -39.2564,  647.8799]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #5: tensor([[ -51.4908, -220.1968, -170.7051, -263.6776, -160.4643, -227.8529,
         -288.4439,  -90.1703, 1473.8951]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #6: tensor([[ -74.8573, -316.6968, -247.4294, -381.7199, -231.4845, -328.1207,
         -416.6414, -130.7230, 2127.4673]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #7: tensor([[ -230.7179,  -973.0297,  -758.7123, -1172.0582,  -710.6502, -1008.8187,
         -1279.6422,  -400.5177,  6533.4443]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #8: tensor([[ -59.8611, -253.8505, -198.4761, -305.7727, -185.5792, -262.5653,
         -334.2016, -105.3695, 1704.9181]], grad_fn=<AddmmBackward>)
tensor([8])
Completed t

tensor([8])
Completed training output for image #47: tensor([[ -82.8323, -351.2461, -274.4306, -423.1810, -255.6939, -363.7109,
         -462.6123, -144.8397, 2359.3237]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #48: tensor([[ -53.1540, -225.0473, -175.2341, -270.6229, -163.6129, -232.3190,
         -296.3896,  -92.7934, 1509.2242]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #49: tensor([[ -63.5107, -267.7026, -208.5584, -322.8617, -195.2312, -276.3901,
         -352.3351, -110.1746, 1795.3629]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #50: tensor([[ -35.0601, -144.8670, -113.4772, -174.8214, -106.6865, -150.5034,
         -191.5688,  -60.2473,  977.7833]], grad_fn=<AddmmBackward>)
tensor([8])
Completed training output for image #51: tensor([[ -39.0397, -162.8648, -127.2998, -195.8665, -119.0858, -169.1380,
         -214.1421,  -67.5171, 1093.5905]], grad_fn=<AddmmBackward>)
tensor([8])
Com

Below we validate our model on our test dataset. We are still calculating the loss and accuracy, but we are not backpropagating the gradients since backpropagation is only required during training. 

In [145]:
limit_num_pictures = 5
vgg16.eval()
testing_loss = 0
testing_accuracy = 0
running_loss = 0.0
num_correct = 0
for i, data in enumerate(test_loader):
    if limit_num_pictures:
        if i > limit_num_pictures:
            break
    inputs, _, path, label = data
    print(label)
    output = vgg16(inputs)
    loss = criterion(output, label)

    running_loss += loss.item()
    _, preds = torch.max(output.data, 1)
    num_correct += (preds == label).sum().item()
    print("Classification for test image #{}: {}".format(i, output))

testing_loss = running_loss/len(test_loader.dataset)
testing_accuracy = 100. * num_correct/len(test_loader.dataset)

tensor([0])
Classification for test image #0: tensor([[ -46.5451, -197.2539, -153.6352, -237.6385, -144.1485, -204.3144,
         -259.8272,  -81.4979, 1325.1384]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #1: tensor([[ -59.7733, -253.4835, -197.3752, -305.3766, -185.2311, -262.7484,
         -333.6786, -104.6743, 1702.0260]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #2: tensor([[ -85.9534, -363.0342, -282.4321, -437.1697, -265.2544, -376.4053,
         -477.7173, -149.5132, 2435.9871]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #3: tensor([[ -84.3488, -355.7207, -276.8194, -428.8018, -259.3821, -368.8505,
         -468.2275, -146.4685, 2389.1001]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #4: tensor([[ -39.7091, -169.0267, -131.5715, -203.6964, -123.2955, -174.8420,
         -222.7304,  -69.9545, 1135.2823]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #5: tens

Classification for test image #43: tensor([[ -53.6603, -225.5631, -175.6736, -271.3678, -165.3616, -233.8936,
         -297.3882,  -92.9696, 1516.1080]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #44: tensor([[ -55.5918, -236.8031, -184.3675, -285.5789, -173.1349, -245.4274,
         -312.1257,  -98.8346, 1591.5121]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #45: tensor([[ -66.6265, -280.7687, -219.7522, -338.1720, -205.1672, -291.3378,
         -369.0779, -115.5954, 1885.8142]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #46: tensor([[-108.0950, -457.3062, -356.0392, -550.2397, -333.8447, -473.8727,
         -600.8261, -188.3554, 3068.6824]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #47: tensor([[ -38.4738, -162.2710, -127.0855, -196.0240, -118.9579, -168.9628,
         -214.4737,  -66.7474, 1093.8141]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #48: tensor([[ 

tensor([0])
Classification for test image #86: tensor([[ -48.2409, -204.3315, -159.1772, -246.7264, -149.1212, -212.7668,
         -269.3133,  -84.6604, 1373.3303]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #87: tensor([[ -63.9365, -271.6682, -211.4374, -326.8429, -198.3840, -281.0849,
         -357.2967, -112.1882, 1823.1431]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #88: tensor([[ -68.1268, -288.0561, -224.6966, -346.7844, -210.7683, -298.9082,
         -379.5454, -118.9052, 1936.2745]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #89: tensor([[ -55.4395, -235.5477, -183.3447, -284.0875, -172.5287, -244.5369,
         -310.2807,  -97.1741, 1583.6761]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #90: tensor([[ -55.4310, -234.6478, -183.3089, -283.2503, -171.2305, -243.5179,
         -308.3966,  -96.5463, 1575.8229]], grad_fn=<AddmmBackward>)
tensor([0])
Classification for test image #91

Choquet Integral

Once we have a pair of values we need to aggregate them together to get an overall ranking. We take the aesthetic and technical ranking derived from the network and determine a fuzzy measure to monitor for uncertainty.